<h2>これはASAのACLをテストするためのノートブックです。</h2>
始めに、以下のセルでBatfishに対する質問を読み込みます。

In [1]:
%run startup.py
load_questions()

次に、以下のセルでshow running-configファイルが格納されている場所を指定します。

In [2]:
# Assign a friendly name to your network and snapshot
NETWORK_NAME = "ASA_acl"
SNAPSHOT_NAME = "ASA_firewall"

SNAPSHOT_PATH = "networks/example-filters/mycurrent"

# Now create the network and initialize the snapshot
bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf_upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf_upload_diagnostics)


'ASA_firewall'

In [3]:
parse_status = bfq.fileParseStatus().answer().frame()

In [4]:
parse_status

,File_Name,Status,Nodes
0,configs/ASA_after_acl_summary.cfg,PARTIALLY_UNRECOGNIZED,['asa_after_acl_summary']
1,configs/ASA_before_acl_summary.cfg,PARTIALLY_UNRECOGNIZED,['asa_before_acl_summary']
2,configs/N9kv-1.cfg,PARTIALLY_UNRECOGNIZED,['n9kv-1']
3,configs/N9kv-2.cfg,PARTIALLY_UNRECOGNIZED,['n9kv-2']
4,configs/after_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['after_summary_asa']
5,configs/before_summary_asa.cfg,PARTIALLY_UNRECOGNIZED,['before_summary_asa']
6,configs/ciscoasa.cfg,PARTIALLY_UNRECOGNIZED,['ciscoasa']


<h3>以下のセルでサマライズされる前のアクセスリスト のテストを行います。</h3>

In [5]:
ip_flow = HeaderConstraints(srcIps='172.20.0.21/24',
                                              dstIps='8.8.8.8')
answer = bfq.testFilters(headers=ip_flow,
                         nodes='asa_before_acl_summary',
                         filters="SPLIT-ACL").answer() 
show(answer.frame())

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,asa_before_acl_summary,SPLIT-ACL,Start Location: asa_before_acl_summarySrc IP: 172.20.0.0Src Port: 49152Dst IP: 8.8.8.8Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


<h3>以下のセルでサマライズされた後のアクセスリスト のテストを行います。</h3>
サマライズ前後で結果が同じかどうか比較してみてください。

In [6]:
ip_flow = HeaderConstraints(srcIps='172.20.0.21/24',
                                              dstIps='8.8.8.8')
answer = bfq.testFilters(headers=ip_flow,
                         nodes='asa_after_acl_summary',
                         filters="SPLIT-ACL").answer() 
show(answer.frame())

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,asa_after_acl_summary,SPLIT-ACL,Start Location: asa_after_acl_summarySrc IP: 172.20.0.0Src Port: 49152Dst IP: 8.8.8.8Dst Port: 80IP Protocol: TCP,DENY,no-match,"AclTrace(events=[AclTraceEvent(class_name='org.batfish.datamodel.acl.DefaultDeniedByIpAccessList', description=""Flow did not match 'standard ipv4 access-list' named 'SPLIT-ACL'"", lineDescription=None)])"


<h4>以下のstartLocationとフィルタの引数は、さまざまな概念を簡単に指定できるようにするBatfish指定子を使用しています。 enter（firewall [Ethernet0 / 0/2]）は、テストしたいパケットが 'Ethernet0 / 0/2'（最初のゾーンのインタフェース）を介して 'firewall'に入ることを指定し、outFilterOf（Ethernet0 / 0/3）はそれを指定します。テストしたいフィルタは 'Ethernet0 / 0/3'（2番目のゾーンのインタフェース）のoutフィルタです。</h4>

In [7]:
# Test if a host can reach the DNS server
http_flow = HeaderConstraints(srcIps="10.114.64.1",
                              dstIps="10.114.60.10",
                              applications=["http"])
answer = bfq.testFilters(headers=http_flow,
                         startLocation="@enter(firewall[GigabitEthernet0/0/2])",
                         filters="@out(GigabitEthernet0/0/3)").answer()
show(answer.frame())

BatfishException: Work terminated abnormally
work_item: {"containerName": "ASA_acl", "id": "2c90fbd0-e44d-47dc-9960-41d00da5c1a8", "requestParams": {"answer": "", "questionname": "__testFilters_39ec51bc-56e5-4023-9e1c-d6d1d6b5932b", "testrig": "ASA_firewall"}, "testrigName": "ASA_firewall"}

log: Loading configurations for NetworkSnapshot{network=0e1d6b10-953d-4b8b-a366-6fa1697fb04e, snapshot=24852f72-4803-40e4-b049-87642427c750}
Loading configurations for NetworkSnapshot{network=0e1d6b10-953d-4b8b-a366-6fa1697fb04e, snapshot=24852f72-4803-40e4-b049-87642427c750}
Loading configurations for NetworkSnapshot{network=0e1d6b10-953d-4b8b-a366-6fa1697fb04e, snapshot=24852f72-4803-40e4-b049-87642427c750}
Loading configurations for NetworkSnapshot{network=0e1d6b10-953d-4b8b-a366-6fa1697fb04e, snapshot=24852f72-4803-40e4-b049-87642427c750}
Loading configurations for NetworkSnapshot{network=0e1d6b10-953d-4b8b-a366-6fa1697fb04e, snapshot=24852f72-4803-40e4-b049-87642427c750}
Loading configurations